## Induce hexapod translations and take CWFS datasets (without correcting) 

Test requires low turbulence (good seeing) ideally.

In [ ]:
import sys
import asyncio
import time
import os

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.idl.enums.Script import ScriptState

from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger

In [ ]:
logger = DecoratedLogger.get_decorated_logger()
logger.level = logging.DEBUG

## Instantiate the CWFS Script

First derive and index for the script that is specific to your user

In [ ]:
logger.info(f'Your UID is {os.getuid()}')
index=os.getuid()*10+np.random.randint(0,9)
logger.info(f'The generated index is {index}')

Instantiate the script then start all remotes

In [ ]:
script = LatissCWFSAlign(index=index, remotes=True)  # this essentially calls the init method
await script.start_task

Set the script to have a DEBUG log level

In [ ]:
script.log.level = logging.DEBUG

## Slew to a target to focus the telescope
This is not part of the actual script but seems to come in handy, so the code is here in the notebook for convenience

Print the available instrument filter and gratings. <br>
This is useful both for slewing and for the configuration steps below

In [ ]:
inst_setup = await script.latiss.get_available_instrument_setup()
logger.info(f'filters are: {inst_setup[0]},\n gratings are: {inst_setup[1]}')

### Set filter

In [ ]:
filt='SDSSg'

### Find/Specify a target based on the Az/El and slew to it

In [ ]:
# Find target
target = await script.atcs.find_target(170,74,mag_limit=9.0)

In [ ]:
# or introduce target name here:
# target = "TIC 181887100"  

### Slew to target, setting the rotator to have azimuth along the X-axis (rows), and elevation along the Y-axis (columns)

In [ ]:
await script.atcs.slew_object(target, rot_type=RotType.Parallactic, rot=90)

Check that the target is in the field

In [ ]:
test_image = await script.latiss.take_engtest(exptime=15, n=1, filter=filt, grating='empty_1')
print ('Check saturation test_image = ' + str(test_image))

### Offset the telescope, set to zero if not required
Offsets are in detector X/Y coordinates and in arcseconds

In [ ]:
xoffset=0; yoffset=0
await script.atcs.offset_xy(x=xoffset, y=yoffset)

## Collimate and Focus the Telescope

## Configure the script

In [ ]:
configuration = yaml.safe_dump({"filter": filt, 
                                "grating": 'empty_1',
                                "exposure_time": 20,})

Set script state to UNCONFIGURED. <br>
The next line is not required the first time the script is run, however, in each additional instance the cell is run, an error will be thrown if it is not included. <br>
Therefore, it is included here despite being a non-operation in the first instance.

In [ ]:
script.set_state(ScriptState.UNCONFIGURED)

## Put the ScriptState to CONFIGURED

In [ ]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

Set these script parameters to None to verify a re-reduction does not happen of the images

In [ ]:
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10

## Set groupID and launch the script
This sets the same group ID for all exposures taken in the script

In [ ]:
group_id_data = script.cmd_setGroupId.DataType(
                groupId=astropy.time.Time.now().isot
            )
await script.do_setGroupId(group_id_data)
await script.arun()

## Telescope is now collimated and in focus

### Grab focus offsets incase we need to come back into focus

In [ ]:
original_focus_offset = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(original_focus_offset)

### Also grab the hexapod offsets - especially the x/y values

In [ ]:
original_hexapod_offset = await script.atcs.rem.ataos.evt_correctionOffsets.aget()
print(original_hexapod_offset)


# Run loop over hexapod translations of the secondary mirror

### Take a test image if desired

In [ ]:
# await script.latiss.take_engtest(exptime=2, n=1, filter=filt,grating='empty_1')

## x-y secondary offset grid
The telescope will offset to compensate for the hexapod offsets so the image stays centered in the detector. 

In [ ]:
# Define here the number of images and exposure times!!
n_of_pairs = 2

In [ ]:
hexapod_offset_scale = [
    [52.459, 0.0, 0.0],
    [0.0, 50.468, 0.0],
    [0.0, 0.0, 0.0],
]

In [ ]:
axes = ['x','y','both', 'both different signs']
axes = ['x']
# offsets = [2.0, 1.0, 0.25, -0.25, -1.0, -2.0]
step=0.15
offsets=np.arange(-2*step, 2.1*step, 0.15) # the 0.1 is so the endpoint is included
print(offsets)

In [ ]:
# Increase to 8 due to mount issues
script.atcs.tel_settle_time=3

In [ ]:
# First grab the current offsets
current_hexapod_offset = await script.atcs.rem.ataos.evt_correctionOffsets.aget()
original_ptg_offsets = await script.atcs.rem.atptg.evt_offsetSummary.aget()

for axis in axes:
    for offset in offsets:
        script.log.debug(f'axis is {axis}, offset is {offset}')
        # Assign hexapod offsets as an array
        if axis == 'x':
            hexapod_offset = [original_hexapod_offset.x-current_hexapod_offset.x+offset,
                              0,
                              0]        
        elif axis == 'y':
            hexapod_offset = [0,
                              original_hexapod_offset.y-current_hexapod_offset.y+offset,
                              0]
        elif axis == 'both': 
            hexapod_offset = [original_hexapod_offset.x-current_hexapod_offset.x+offset,
                              original_hexapod_offset.y-current_hexapod_offset.y+offset,
                              0]
        elif axis == 'both different signs':
            hexapod_offset = [original_hexapod_offset.x-current_hexapod_offset.x+offset,
                              -(original_hexapod_offset.y-current_hexapod_offset.y+offset),
                              0]
        else:
            raise RuntimeError(f'Specified axis [{axis}] is invalid')
        
        # Apply the pointing offset
        await script.atcs.rem.ataos.cmd_offset.set_start(x=hexapod_offset[0], y=hexapod_offset[1])

        script.log.debug(f'Hexapod offsets from the current position to the target are: {hexapod_offset} [mm]')

       
        # Calculating telescope az and elevation offsets to keep image centered.         
        tel_offset = np.matmul(hexapod_offset, hexapod_offset_scale) 

        script.log.debug(f'Telescope will be offset {tel_offset[1]} arcsec in azimuth ' 
              f'and {tel_offset[0]} arcsec in elevation relative to the'
               'starting position.')
        await script.atcs.offset_azel(az=tel_offset[1], el=tel_offset[0],
                                      relative=True, persistent=False)

        # Take a cwfs pair
        script.log.debug(f'Taking a CWFS pair')
        await script.take_intra_extra()
        
# Send hexapod to original position
script.log.info('CWFS_SENS_TEST:Loop Completed Sending hexapod to original position and send back to the original offsets')
await script.atcs.offset_xy(xoffset, yoffset)
await script.atcs.rem.ataos.cmd_offset.set_start(x=original_hexapod_offset.x-current_hexapod_offset.x,
                                                 y=original_hexapod_offset.y-current_hexapod_offset.y)